<a href="https://colab.research.google.com/github/whossrinidhi/echoverify/blob/main/final_deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: make a model to detect audio is fake or real

!pip install librosa soundfile

import os
import librosa
import soundfile as sf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohammedabdeldayem/the-fake-or-real-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/the-fake-or-real-dataset


In [ ]:
import os

base_path = "/kaggle/input/the-fake-or-real-dataset"

# List contents
for root, dirs, files in os.walk(base_path):
    print(root)
    for dir_name in dirs:
        print("  DIR:", dir_name)



/kaggle/input/the-fake-or-real-dataset
  DIR: for-norm
  DIR: for-original
  DIR: for-2sec
  DIR: for-rerec
/kaggle/input/the-fake-or-real-dataset/for-norm
  DIR: for-norm
/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm
  DIR: validation
  DIR: training
  DIR: testing
/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/validation
  DIR: fake
  DIR: real
/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/validation/fake
/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/validation/real
/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training
  DIR: fake
  DIR: real
/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/fake


KeyboardInterrupt: 

In [ ]:
real_audio_path = "/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/real"
fake_audio_path = "/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake"



In [ ]:
# Load audio files and extract features
def extract_features(file_path):
  audio, sample_rate = librosa.load(file_path, sr=None)
  mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
  return np.mean(mfccs.T, axis=0)

# Load real and fake audio samples (replace with your actual file paths)
# Define the paths for real and fake audio files


# Gather all the .wav files from the directories
real_audio_files = [os.path.join(real_audio_path, file) for file in os.listdir(real_audio_path) if file.endswith('.wav')]
fake_audio_files = [os.path.join(fake_audio_path, file) for file in os.listdir(fake_audio_path) if file.endswith('.wav')]

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/real'

In [ ]:
# Extract features and create labels
X = []
y = []
for file in real_audio_files:
  X.append(extract_features(file))
  y.append(0)  # Label 0 for real audio

for file in fake_audio_files:
  X.append(extract_features(file))
  y.append(1)  # Label 1 for fake audio

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

KeyboardInterrupt: 

In [ ]:
# Train a Random Forest classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9641833810888252


In [ ]:
# Function to predict if a new audio file is fake or real
def predict_fake_audio(file_path):
  features = extract_features(file_path)
  prediction = model.predict([features])[0]
  if prediction == 0:
    return "Real audio"
  else:
    return "Fake audio"

In [ ]:
# Example usage:
new_audio_file = input("Enter the path to the audio file you want to check: ")

# Ensure that the file exists
if os.path.exists(new_audio_file):
    result = predict_fake_audio(new_audio_file)
    print(f"The prediction for the audio file is: {result}")
else:
    print("The specified file does not exist. Please check the path and try again.")

Enter the path to the audio file you want to check: /root/.cache/kagglehub/datasets/mohammedabdeldayem/the-fake-or-real-dataset/versions/2/for-2sec/for-2seconds/testing/real/file10.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav
The prediction for the audio file is: Real audio


In [ ]:
!pip install librosa soundfile

import librosa
import soundfile as sf
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Function to add noise to an audio sample (data augmentation)
def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    return augmented_audio

In [ ]:
# Load audio files and extract features
def extract_features(file_path, augment=False):
    audio, sample_rate = librosa.load(file_path, sr=None)
    if augment:
        audio = add_noise(audio)  # Apply noise augmentation

    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
    spec_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)

    features = np.hstack([
        np.mean(mfccs.T, axis=0),
        np.mean(chroma.T, axis=0),
        np.mean(spec_contrast.T, axis=0)
    ])
    return features

In [ ]:
# Define the paths for real and fake audio files
base_path = '/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds'

train_real_path = os.path.join(base_path, 'training/real')
train_fake_path = os.path.join(base_path, 'training/fake')
test_real_path = os.path.join(base_path, 'testing/real')
test_fake_path = os.path.join(base_path, 'testing/fake')

# Gather all the .wav files from the directories
train_real_files = [os.path.join(train_real_path, file) for file in os.listdir(train_real_path) if file.endswith('.wav')]
train_fake_files = [os.path.join(train_fake_path, file) for file in os.listdir(train_fake_path) if file.endswith('.wav')]
test_real_files = [os.path.join(test_real_path, file) for file in os.listdir(test_real_path) if file.endswith('.wav')]
test_fake_files = [os.path.join(test_fake_path, file) for file in os.listdir(test_fake_path) if file.endswith('.wav')]

In [ ]:
# Extract features and create labels
X_train, y_train, X_test, y_test = [], [], [], []

for file in train_real_files:
    X_train.append(extract_features(file))
    X_train.append(extract_features(file, augment=True))  # Augmented data
    y_train.append(0)  # Label 0 for real audio
    y_train.append(0)  # Label 0 for augmented real audio

for file in train_fake_files:
    X_train.append(extract_features(file))
    X_train.append(extract_features(file, augment=True))  # Augmented data
    y_train.append(1)  # Label 1 for fake audio
    y_train.append(1)  # Label 1 for augmented fake audio

for file in test_real_files:
    X_test.append(extract_features(file))
    y_test.append(0)  # Label 0 for real audio

for file in test_fake_files:
    X_test.append(extract_features(file))
    y_test.append(1)  # Label 1 for fake audio

/usr/local/lib/python3.11/dist-packages/librosa/core/pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [ ]:
# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model with additional metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.5873
Precision: 0.8417
Recall: 0.2151
F1-score: 0.3426


In [ ]:
# Function to predict if a new audio file is fake or real
def predict_fake_audio(file_path):
    features = extract_features(file_path)
    prediction = model.predict([features])[0]
    if prediction == 0:
        return "Real audio"
    else:
        return "Fake audio"

In [ ]:
# Example usage:
new_audio_file = input("Enter the path to the audio file you want to check: ")

# Ensure that the file exists
if os.path.exists(new_audio_file):
    result = predict_fake_audio(new_audio_file)
    print(f"The prediction for the audio file is: {result}")
else:
    print("The specified file does not exist. Please check the path and try again.")

Enter the path to the audio file you want to check: /root/.cache/kagglehub/datasets/mohammedabdeldayem/the-fake-or-real-dataset/versions/2/for-2sec/for-2seconds/testing/real/file10.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav
The prediction for the audio file is: Real audio


In [ ]:
import joblib

# Save the trained model
joblib.dump(model, 'random_forest.pkl')

# Optional: Download the model to your Mac
from google.colab import files
files.download('random_forest.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save the model in Drive
joblib.dump(model, '/content/drive/MyDrive/random_forest.pkl')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Function to predict if a new audio file is fake or real
def predict_fake_audio(file_path):
    features = extract_features(file_path)
    prediction = model.predict([features])[0]
    if prediction == 0:
        return "Real audio"
    else:
        return "Fake audio"

In [ ]:
# Example usage:
new_audio_file = input("Enter the path to the audio file you want to check: ")

# Ensure that the file exists
if os.path.exists(new_audio_file):
    result = predict_fake_audio(new_audio_file)
    print(f"The prediction for the audio file is: {result}")
else:
    print("The specified file does not exist. Please check the path and try again.")

Enter the path to the audio file you want to check: /kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/testing/real/file10.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav
The prediction for the audio file is: Real audio


In [ ]:
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
import os
print(os.listdir())  # This will list all files in the current working directory


In [ ]:
import joblib

model = joblib.load('random_forest.pkl')  # Use the exact filename from the upload
print("Model loaded successfully!")


In [ ]:
# Function to predict if a new audio file is fake or real
def predict_fake_audio(file_path):
    features = extract_features(file_path)
    prediction = model.predict([features])[0]
    if prediction == 0:
        return "Real audio"
    else:
        return "Fake audio"

In [ ]:

new_audio_file = input("Enter the path to the audio file you want to check: ")

# Ensure that the file exists
if os.path.exists(new_audio_file):
    result = predict_fake_audio(new_audio_file)
    print(f"The prediction for the audio file is: {result}")
else:
    print("The specified file does not exist. Please check the path and try again.")

Enter the path to the audio file you want to check: /root/.cache/kagglehub/datasets/mohammedabdeldayem/the-fake-or-real-dataset/versions/2/for-2sec/for-2seconds/testing/real/file10.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav


NameError: name 'os' is not defined

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import numpy as np
import librosa
import joblib  # Load RandomForest model
import os



# Extract features ensuring we return 32 features
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, sr=None)

    # Extract MFCCs (13 features)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    mfccs = np.mean(mfccs.T, axis=0)

    # Extract Chroma features (12 features)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
    chroma = np.mean(chroma.T, axis=0)

    # Extract Spectral Contrast (7 features)
    spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
    spectral_contrast = np.mean(spectral_contrast.T, axis=0)

    # Combine all extracted features to match 32 features
    features = np.hstack([mfccs, chroma, spectral_contrast])

    return features  # Now it's a 1D array with 32 features

# Function to predict if the audio is fake or real
def predict_fake_audio(file_path):
    try:
        features = extract_features(file_path)  # Extract features
        prediction = model.predict([features])[0]  # Ensure correct shape

        return "Real audio" if prediction == 0 else "Fake audio"
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio UI
interface = gr.Interface(
    fn=predict_fake_audio,
    inputs=gr.Audio(type="filepath"),  # Accepts a .wav file path
    outputs="text",
    title="Fake Audio Detection",
    description="Upload a .wav file to check if it's real or fake using a Random Forest model."
)

# Launch Gradio app
interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d22bba6430d27212f7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
/root/.cache/kagglehub/datasets/mohammedabdeldayem/the-fake-or-real-dataset/versions/2/for-2sec/for-2seconds/testing/real/file10.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav


In [ ]:
import gradio as gr
import numpy as np
import librosa
import joblib  # Load RandomForest model


# Extract features ensuring we return 32 features
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, sr=None)

    # Extract MFCCs (13 features)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    mfccs = np.mean(mfccs.T, axis=0)

    # Extract Chroma features (12 features)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
    chroma = np.mean(chroma.T, axis=0)

    # Extract Spectral Contrast (7 features)
    spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
    spectral_contrast = np.mean(spectral_contrast.T, axis=0)

    # Combine all extracted features to match 32 features
    features = np.hstack([mfccs, chroma, spectral_contrast])

    return features  # Now it's a 1D array with 32 features


# Function to predict if the audio is fake or real
def predict_fake_audio(file_obj):
    try:
        file_path = file_obj.name  # Extract file path from Gradio's File object
        features = extract_features(file_path)  # Extract features
        prediction = model.predict([features])[0]  # Ensure correct shape

        return "Real audio" if prediction == 0 else "Fake audio", file_path
    except Exception as e:
        return f"Error: {str(e)}", None


# Gradio UI (File Upload + Audio Playback)
interface = gr.Interface(
    fn=predict_fake_audio,
    inputs=gr.File(type="filepath"),  # File upload only (no recording)
    outputs=["text", gr.Audio()],  # Text prediction + Audio playback
    title="Fake Audio Detection",
    description="Upload a .wav file to check if it's real or fake using a Random Forest model."
)

# Launch Gradio app
interface.launch()



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a507a3ec1e9473f3c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
print("hello")

hello


In [ ]:
/testing/real/file10.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav